# Cloud Pak for Data Dashboard - Continuous Update
This notebook will run continuously through all of the stocks in the database. The Dashboard will be updated as quickly as possible.

In [1]:
%run db2.ipynb
%run connection.ipynb
%run dashboard.ipynb
seturl('http://tipboard:7272')

Db2 Extensions Loaded.
Connection successful.


### Select a Stock Symbol from the following list
Select one main stock and two others to compare against that are currently available from the following list.

In [2]:
all_stocks = %sql -array SELECT TRIM(SYMBOL) FROM TRADING.STOCK_SYMBOLS WHERE TRIM(SYMBOL) != 'DJIA' \
                         ORDER BY SYMBOL 
all_names  = %sql -array SELECT TRIM(COMPANY) FROM TRADING.STOCK_SYMBOLS WHERE TRIM(SYMBOL) != 'DJIA' \
                         ORDER BY SYMBOL

Define the block of code that will populate the dashboard.

# Publishing Dashboard

In [3]:
def publish_dashboard(stock1_in, stock2_in, stock3_in, stock_name_in):
    
    stock1 = stock1_in
    stock2 = stock2_in
    stock3 = stock3_in
    weeks = [25, 45]
    stock_name = stock_name_in
    
    # Step 2: Get the open price of the stock and the volume for the first record found
    stock_values = %sql SELECT OPEN, CLOSE, VOLUME FROM TRADING.STOCK_HISTORY \
                        WHERE SYMBOL = :stock1 FETCH FIRST ROW ONLY
    stock_open = str(stock_values['OPEN'][0])
    stock_current = str(stock_values['CLOSE'][0])
    stock_volume = int(stock_values['VOLUME'][0])

    # Step 3: Publish the information into cell STOCK_INFO
    publishText('stock_info',stock1,stock_name,stock_current,'Open',stock_open,'Volume',stock_volume)

    # Step 4: Get the YTD stock values on a weekly basis
    sql = '''
    WITH WEEKLY(SYMBOL, WEEKNO, VALUE) AS (
       SELECT SYMBOL, WEEK(TX_DATE), DEC(AVG(OPEN),9,2)
         FROM TRADING.STOCK_HISTORY
         WHERE SYMBOL IN (:stock1, :stock2, :stock3) 
               AND TX_DATE != '2017-12-01' 
               AND WEEK(TX_DATE) BETWEEN :weeks[0] AND :weeks[1]
         GROUP BY SYMBOL, WEEK(TX_DATE)
    )
    SELECT * FROM WEEKLY 
    ORDER BY SYMBOL, WEEKNO
    '''
    data = %sql -array {sql}

    stock1_volume = []
    stock2_volume = []
    stock3_volume = []
    for row in data[1:]:
        if (row[0] == stock1): stock1_volume.append([row[1],row[2]])
        if (row[0] == stock2): stock2_volume.append([row[1],row[2]])
        if (row[0] == stock3): stock3_volume.append([row[1],row[2]])


    # Step 5: Publish the data into the cell STOCK_YTD
    publishLine('stock_ytd','{0} vs {1} vs {2}'.format(stock1,stock2,stock3),'Stock Price Average Last 20 Weeks', 
                [stock1,stock2,stock3],[stock1_volume, stock2_volume, stock3_volume])

    # Step 6: Top Purchase/Sell of Stock today (Note: All Stocks)
    
    sql = '''
    WITH
    TX_DAY(TX_DATE) AS (
      VALUES NEXT_DAY(DATE('2018-01-01') + INT(RAND()*350) DAYS, 'Thursday')
    ),
    PURCHASED(CUSTID,AMOUNT) AS (
      SELECT CUSTID, SUM(QUANTITY) AS AMOUNT
        FROM TRADING.STOCK_TRANSACTIONS ST, TX_DAY TX
      WHERE ST.TX_DATE = TX.TX_DATE 
      GROUP BY CUSTID
    )
    SELECT C.LASTNAME, C.CITY, C.STATE, P.AMOUNT
      FROM PURCHASED P, TRADING.CUSTOMER C
      WHERE C.CUSTID = P.CUSTID
    ORDER BY ABS(P.AMOUNT) DESC
    FETCH FIRST 9 ROWS ONLY 
    '''
    data = %sql -array {sql}
    customer = []
    purchase = []
    location = []
    color    = []
    
    row_count = 0
    for row in data[1:]:
        row_count = row_count + 1
        customer.append(row[0])
        
        if (int(row[3]) < 0):
            color.append("red")
        else:
            color.append("green")
            
        purchase.append(row[3])
        location.append("{0}, {1}".format(row[1],row[2]))
    
    if (row_count == 0): # No data found
        purchase.append(0)
        customer.append("No stock trades completed")
        color.append("blue")
        location.append("")
    
    # Step 7: Publish the result to the cell STOCK_COMPARE    
    publishListing('stock_purchase',purchase,color,customer,location)

    # Step 8: Compare the stocks by capitalization

    sql = '''
    WITH WEEK45(SYMBOL, PURCHASES) AS (
      SELECT SYMBOL, SUM(VOLUME * CLOSE) FROM TRADING.STOCK_HISTORY
        WHERE WEEK(TX_DATE) =  45 AND SYMBOL IN (:stock1, :stock2, :stock3)
      GROUP BY SYMBOL
    )
    SELECT SYMBOL, PURCHASES FROM WEEK45
    ORDER BY SYMBOL, PURCHASES
    '''

    data = %sql -array {sql}

    # Step 9: Publish the result to the cell TOP_STOCKS   

    publishPie('top_stocks','Stock Capitalization',data[1:])

    # Step 10: 30 Day Moving Average with trend line

    sql = '''
    WITH MOVAVG(WEEKNO, OPEN, MOVING) AS (
       SELECT WEEK(TX_DATE) AS WEEK, OPEN, 
         AVG(OPEN) OVER (
           ORDER BY TX_DATE
         ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING) AS MOVING_AVG
      FROM TRADING.STOCK_HISTORY
         WHERE SYMBOL = :stock1
      ORDER BY WEEK(TX_DATE)
    )
    SELECT WEEKNO, DEC(AVG(OPEN),9,2), DEC(AVG(MOVING),9,2) FROM MOVAVG
      WHERE WEEKNO BETWEEN 10 AND 20
    GROUP BY WEEKNO
    ORDER BY WEEKNO
    '''
    data = %sql -array {sql}

    stock1_volume = []
    stock2_volume = []
    for row in data[1:]:
        stock1_volume.append([row[0],row[1]])
        stock2_volume.append([row[0],row[2]])
    
    # Step 11: Publish the trend line to STOCK_30DAY
    
    publishLine('stock_30day',stock1,'30-day Moving Average',['Base','Trend'],[stock1_volume,stock2_volume])

Run the code forever, or until you hit stop.

In [4]:
import random
import time

stock_names = []
for stock in all_stocks[1:]:
    stock_names.append(stock[0])
    
while True:
    random_stocks = random.sample(all_stocks[1:],3)
    stock1 = random_stocks[0][0]
    stock2 = random_stocks[1][0]
    stock3 = random_stocks[2][0]
    
    print('Stocks: {0}, {1}, {2}'.format(stock1,stock2,stock3))

    idx = stock_names.index(stock1)
    stock_name = all_names[idx+1][0]
    
    publish_dashboard(stock1, stock2, stock3, stock_name)   
    time.sleep(1)

Stocks: NKE, CSCO, UNH
Stocks: INTC, CAT, UNH
Stocks: CSCO, CAT, NKE
Stocks: JPM, V, WMT
Stocks: CVX, PG, XOM
Stocks: JPM, UTX, MSFT
Stocks: MRK, JNJ, VZ
Stocks: PG, DIS, UNH
Stocks: MRK, HD, MCD
Stocks: NKE, CVX, JNJ
Stocks: MSFT, AXP, GS
Stocks: MSFT, NKE, AXP
Stocks: CSCO, PFE, KO
Stocks: WMT, V, DIS
Stocks: GS, CVX, UNH
Stocks: AAPL, CSCO, AXP
Stocks: UTX, AAPL, WMT
Stocks: BA, WMT, XOM
Stocks: CVX, AAPL, DIS
Stocks: XOM, WBA, BA
Stocks: PG, VZ, AAPL
Stocks: CSCO, MRK, XOM
Stocks: WMT, JPM, XOM
Stocks: CVX, JPM, GS
Stocks: CVX, CSCO, MMM
Stocks: TRV, DIS, MMM
Stocks: WMT, MSFT, XOM
Stocks: AAPL, UTX, XOM
Stocks: MMM, INTC, MCD
Stocks: KO, IBM, HD
Stocks: KO, GS, JPM
Stocks: MCD, AXP, CSCO
Stocks: KO, V, VZ
Stocks: CVX, UNH, MSFT
Stocks: KO, PG, AAPL
Stocks: DWDP, WBA, MSFT
Stocks: TRV, JNJ, PG
Stocks: UNH, JNJ, KO
Stocks: AAPL, CAT, MMM
Stocks: DIS, MCD, IBM
Stocks: VZ, IBM, NKE
Stocks: AXP, MSFT, MRK
Stocks: JPM, WMT, BA
Stocks: UTX, AXP, PFE
Stocks: HD, JNJ, V
Stocks: V, PFE, TRV

TypeError: 'NoneType' object is not subscriptable